In [1]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt 
import numpy as np 
import os 
import pandas as pd 
import keras
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
from matplotlib.image import imread
from PIL import Image

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
import tensorflow as tf
import re

from sklearn.model_selection import KFold


#print how many rows and columns are there in the csv file
df = pd.read_csv('train-balanced-sarcasm.csv')


df_useful_x = df[['comment','label']]
df_useful_y = df[['label']]
df_useful_x['comment'] = df_useful_x['comment'].astype('str')

df_useful_x.info()

Using TensorFlow backend.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1010826 entries, 0 to 1010825
Data columns (total 2 columns):
comment    1010826 non-null object
label      1010826 non-null int64
dtypes: int64(1), object(1)
memory usage: 15.4+ MB


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [2]:
df_useful_x

,comment,label
0,NC and NH.,0
1,You do know west teams play against west teams...,0
2,"They were underdogs earlier today, but since G...",0
3,"This meme isn't funny none of the ""new york ni...",0
4,I could use one of those tools.,0
...,...,...
1010821,I'm sure that Iran and N. Korea have the techn...,1
1010822,"whatever you do, don't vote green!",1
1010823,Perhaps this is an atheist conspiracy to make ...,1
1010824,The Slavs got their own country - it is called...,1


In [3]:
df_useful_x['pre_clean_len'] = [len(t) for t in df_useful_x.comment]
df_useful_x

df_useful_x[df_useful_x.pre_clean_len > 140].head(10)
df_useful_x.count

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


<bound method DataFrame.count of                                                    comment  label  \
0                                               NC and NH.      0   
1        You do know west teams play against west teams...      0   
2        They were underdogs earlier today, but since G...      0   
3        This meme isn't funny none of the "new york ni...      0   
4                          I could use one of those tools.      0   
...                                                    ...    ...   
1010821  I'm sure that Iran and N. Korea have the techn...      1   
1010822                 whatever you do, don't vote green!      1   
1010823  Perhaps this is an atheist conspiracy to make ...      1   
1010824  The Slavs got their own country - it is called...      1   
1010825  values, as in capitalism .. there is good mone...      1   

         pre_clean_len  
0                   10  
1                   74  
2                  121  
3                   60  
4            

In [4]:
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))

def text_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()



#nums = [0,1010826]
nums = [0,1010826]
print ("Cleaning and parsing the comments...\n")
clean_comment_texts = []
for i in range(nums[0],nums[1]):
    if( (i+1)%10000 == 0 ):
        print ("comments %d of %d has been processed" % ( i+1, nums[1] ))                                                                    
    clean_comment_texts.append(text_cleaner(df_useful_x['comment'][i]))
    
clean_df = pd.DataFrame(clean_comment_texts,columns=['comment'])

clean_df.head()


Cleaning and parsing the comments...

comments 10000 of 1010826 has been processed
comments 20000 of 1010826 has been processed


/opt/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:294: UserWarning: "b'/'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


comments 30000 of 1010826 has been processed
comments 40000 of 1010826 has been processed
comments 50000 of 1010826 has been processed
comments 60000 of 1010826 has been processed
comments 70000 of 1010826 has been processed
comments 80000 of 1010826 has been processed
comments 90000 of 1010826 has been processed
comments 100000 of 1010826 has been processed
comments 110000 of 1010826 has been processed
comments 120000 of 1010826 has been processed
comments 130000 of 1010826 has been processed
comments 140000 of 1010826 has been processed


/opt/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:294: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


comments 150000 of 1010826 has been processed
comments 160000 of 1010826 has been processed
comments 170000 of 1010826 has been processed
comments 180000 of 1010826 has been processed
comments 190000 of 1010826 has been processed
comments 200000 of 1010826 has been processed
comments 210000 of 1010826 has been processed
comments 220000 of 1010826 has been processed
comments 230000 of 1010826 has been processed
comments 240000 of 1010826 has been processed
comments 250000 of 1010826 has been processed
comments 260000 of 1010826 has been processed
comments 270000 of 1010826 has been processed
comments 280000 of 1010826 has been processed
comments 290000 of 1010826 has been processed
comments 300000 of 1010826 has been processed
comments 310000 of 1010826 has been processed
comments 320000 of 1010826 has been processed
comments 330000 of 1010826 has been processed
comments 340000 of 1010826 has been processed
comments 350000 of 1010826 has been processed
comments 360000 of 1010826 has bee

...

/opt/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:357: UserWarning: "http://localhost:88/emblem-bot/index.php" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


comments 860000 of 1010826 has been processed
comments 870000 of 1010826 has been processed
comments 880000 of 1010826 has been processed
comments 890000 of 1010826 has been processed
comments 900000 of 1010826 has been processed
comments 910000 of 1010826 has been processed
comments 920000 of 1010826 has been processed
comments 930000 of 1010826 has been processed


/opt/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:357: UserWarning: "http://last.word/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


comments 940000 of 1010826 has been processed
comments 950000 of 1010826 has been processed
comments 960000 of 1010826 has been processed
comments 970000 of 1010826 has been processed
comments 980000 of 1010826 has been processed
comments 990000 of 1010826 has been processed
comments 1000000 of 1010826 has been processed
comments 1010000 of 1010826 has been processed


,comment
0,nc and nh
1,you do know west teams play against west teams...
2,they were underdogs earlier today but since gr...
3,this meme isn t funny none of the new york nig...
4,i could use one of those tools


In [13]:
clean_df

sent = [row.split(',') for row in clean_df['comment']]


In [14]:
sent

[['nc and nh'],
 ['you do know west teams play against west teams more than east teams right'],
 ['they were underdogs earlier today but since gronk s announcement this afternoon the vegas line has moved to patriots'],
 ['this meme isn t funny none of the new york nigga ones are'],
 ['i could use one of those tools'],
 ['i don t pay attention to her but as long as she s legal i wouldn t kick her out of bed before she took a load'],
 ['trick or treating in general is just weird'],
 ['blade mastery masamune or gtfo'],
 ['you don t have to you have a good build buy games or save it'],
 ['i would love to see him at lolla'],
 ['i think a significant amount would be against spending their tax dollars on other people'],
 ['damn i was hoping god was real'],
 ['they have an agenda'],
 ['great idea'],
 ['ayy bb wassup it makes a bit more sense in context'],
 ['what the fuck'],
 ['noted'],
 ['because it s what really bothers him and it s a sign of weakness'],
 ['why you fail me my precious'],
 ['

In [17]:
import gensim 
from gensim.models import Word2Vec 
  
model = Word2Vec(sent, min_count=5,size= 500,workers=3, window =3, sg = 1)

In [21]:
from gensim.test.utils import common_texts, get_tmpfile
path = get_tmpfile("word2vec.model")
model.save("word2vec.model")

In [6]:
#show the columns to see which ones we would need to use, and make a subdataframe with columns we need

# convert each dataframe into 2D numpy arrays
#x= clean_df.to_numpy()
#np.asarray(X_train_seq)
#y= df_useful_y.to_numpy()

In [9]:
clean_df.columns

Index(['comment'], dtype='object')

NameError: name 'x' is not defined

In [18]:
from sklearn.model_selection import train_test_split

X_train,X_test, Y_train, Y_test = train_test_split(clean_df,df_useful_y,test_size=0.2,random_state=0)

In [19]:
X_train

,comment
329449,looks more deformed than half elephant
850724,if you want our president to be killed you are...
270451,acceptance
486031,ms just wants to make sure that we don t go ov...
308005,no no no you don t agree with me ergo you are ...
...,...
963395,quad decks the memorials will survive a very s...
117952,i think you mean top
435829,agreed this vid shows how much better this guy...
305711,hang ingat hang mat salleh ka nyanyi lagu ongp...


In [20]:
X_test

,comment
83725,das racist
874445,ugh people are always complaining about us ers
140897,but using the lightning port means that there ...
333371,goes to bar finds pokestop at bar so that s wh...
496236,the only thing wrong with this picture is that...
...,...
626411,i play on a mostly survival server we did have...
626606,i ll hit nyc first
873104,da buon padre fa la cosa giusta e avvisalo o q...
885455,why don t you let the parent do the parenting


In [21]:
Y_train

,label
329449,1
850724,0
270451,0
486031,1
308005,1
...,...
963395,0
117952,0
435829,1
305711,1


In [22]:
Y_test

,label
83725,0
874445,1
140897,0
333371,0
496236,1
...,...
626411,0
626606,0
873104,1
885455,0


In [23]:
print('Reddit Dataset Shape:')
print('X_train: ' + str(X_train.shape))
print('Y_train: ' + str(Y_train.shape))
print('X_test:  '  + str(X_test.shape))
print('Y_test:  '  + str(Y_test.shape))

Reddit Dataset Shape:
X_train: (808660, 1)
Y_train: (808660, 1)
X_test:  (202166, 1)
Y_test:  (202166, 1)


In [24]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras import models
from keras import layers
max_words=5000


tk = Tokenizer(num_words=max_words,
filters='!"#$%&()*+,-./:;<=>?@[\]^_`{"}~\t\n',lower=True, split=" ")
tk.fit_on_texts(X_train)
X_train_seq = tk.texts_to_sequences(X_train)
X_test_seq = tk.texts_to_sequences(X_test)


# pad upto max_words
#X_train = sequence.pad_sequences(X_train, maxlen=max_words)
#X_test = sequence.pad_sequences(X_test, maxlen=max_words)

In [25]:
#X_train=sequence.pad_sequences(X_train, maxlen=max_words)
X_train_seq

[[1]]

In [28]:
MAX_LEN = 500 
X_test_seq
X_train_seq_trunc = pad_sequences(X_train_seq, maxlen=MAX_LEN)
X_test_seq_trunc = pad_sequences(X_test_seq, maxlen=MAX_LEN)

In [30]:
le = LabelEncoder()
y_train_le = le.fit_transform(Y_train)
y_test_le = le.transform(Y_test)
y_train_oh = to_categorical(y_train_le)
y_test_oh = to_categorical(y_test_le)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [31]:
X_train_emb, X_valid_emb, y_train_emb, y_valid_emb = train_test_split(X_train_seq_trunc, y_train_oh, test_size=0.1, random_state=37)

ValueError: Found input variables with inconsistent numbers of samples: [1, 808660]